# Model 040 BasicCh: Coupled Steady-State Solution

This model shows example usage of the Basic model from the TerrainBento package.

Instead of the linear flux law for hillslope erosion and transport, BasicCh uses a nonlinear (cubic) law:

$\frac{\partial \eta}{\partial t} = - KA^{1/2}S - \nabla q_h$ 

$q_h = -D \left( \nabla \eta + \frac{1}{S_c^2} \left[ \nabla \eta \right]^3 \right)$

where $S_c$ is a critical slope gradient. 

Refer to [REF MANUSCRIPT] for the definition of all symbols. 

This notebook (a) shows the initialization and running of this model, (b) saves a NetCDF file of the topography, which we will use to make an oblique Paraview image of the landscape, and (c) creates a slope-area plot at steady state.

In [ ]:
# import required modules
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 20})
%matplotlib inline

from landlab import imshow_grid
from landlab.io.netcdf import write_netcdf

from terrainbento import BasicCh

In [ ]:
# create the parameter dictionary needed to instantiate the model 
# all parameters used are specified in this notebook block

params = {'number_of_node_rows' : 100,
         'number_of_node_columns' : 160,
         'node_spacing' : 10.0, # meters
         'east_boundary_closed' : False,
         'north_boundary_closed' : False,
         'west_boundary_closed' : False,
         'south_boundary_closed' :False,
         'dt' : 10, # meters
         'K_sp' : 0.001, # years^-1
         'm_sp' : 0.5, # unitless
         'n_sp' : 1.0, # unitless
         'linear_diffusivity' : 0.1, # meters^2/year
         'slope_crit' : 0.3, # unitless
         'outlet_lowering_rate' : 0.0005, # meters/year
         'output_filename': 'model_040_output',
         'random_seed': 4897 # set to initialize the topography with reproducible random noise
         }

In [ ]:
# initialize the model by passing the parameter dictionary
model = BasicCh(params=params)

# set a tolerance for determining steady state
tolerance = 0.0001

In [ ]:
# run the model, it will take 190,000 years with the original (160x100) grid

# set dt and initialize the elapsed_time variable
dt = params['dt']
elapsed_time = 0 

# initialize a variable to identify if steady state has been reached
keep_running = True

# run the model until stead state has been reached. 
while keep_running == True:
    
    # save the pre-model-step topography of the core nodes to assess steady state. 
    pre_topo = model.grid.at_node['topographic__elevation'][model.grid.core_nodes]
    
    # run the model one step
    model.run_one_step(dt)
    
    # save the post-model step topography of the core nodes to assess steady state. 
    post_topo = model.grid.at_node['topographic__elevation'][model.grid.core_nodes]
    
    # increment elapsed time
    elapsed_time += dt
    
    # assess if steady state has been reached. 
    if max(abs(pre_topo - post_topo)) <= tolerance: # 1 mm
        keep_running = False
    
    # if time rounds to 10,000 years, print out a log message. 
    if elapsed_time % 10000 == 0:
        print('Elapsed time: ', elapsed_time)
        print('Maximum Topographic Change/Tolerance: ', np.round(max(abs(pre_topo - post_topo))/tolerance, decimals=2))       

In [ ]:
# MAKE SLOPE-AREA PLOT

# plot nodes that are not on the boundary or adjacent to it
plotting_nodes = ((model.grid.node_x > params['node_spacing'])&
                  (model.grid.node_x < params['node_spacing']*(params['number_of_node_columns']-2))&
                  (model.grid.node_y > params['node_spacing'])&
                  (model.grid.node_y < params['node_spacing']*(params['number_of_node_rows']-2)))

# assign area_array and slope_array
area_array = model.grid.at_node['drainage_area'][plotting_nodes]
slope_array = model.grid.at_node['topographic__steepest_slope'][plotting_nodes]

# instantiate figure and plot
fig = plt.figure(figsize=(6, 3.75))
slope_area = plt.subplot()

# plot the data
slope_area.scatter(area_array, slope_array, marker='o', c='k', 
                   label = 'Model BasicCh') 
                   
# make axes log and set limits
slope_area.set_xscale('log')
slope_area.set_yscale('log')

slope_area.set_xlim(9*10**1, 3*10**5)
slope_area.set_ylim(1e-3, 1e-1)

# set x and y labels
slope_area.set_xlabel(r'Drainage area [m$^2$]')
slope_area.set_ylabel('Channel slope [-]')
slope_area.legend(scatterpoints=1,prop={'size':12})
slope_area.tick_params(axis='x', which='major', pad=7)

# save out an output figure
output_figure = os.path.join('output_figures/maintext_taylor_hillslopes_slope_area.eps')
fig.savefig(output_figure, bbox_inches='tight', dpi=1000) # save figure

In [ ]:
#export NetCDF for Paraview image of topography
output_file = os.path.join('output_netcdfs/maintext_taylor_hillslopes.nc')
write_netcdf(output_file, model.grid, format='NETCDF3_64BIT', names='topographic__elevation')

In [ ]:
# make a plot of the final steady state topography
imshow_grid(model.grid, 'topographic__elevation')